In [ ]:
!pip install transformers
!pip install accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.3 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("tinkoff-ai/ruDialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("tinkoff-ai/ruDialoGPT-small").to(device)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

data = pd.read_csv("data.csv")

with open("text_data.txt", "w", encoding="utf-8") as file:
    for index, row in data.iterrows():
        context_3 = str(row['context_3']) if not pd.isna(row['context_3']) else ''
        context_2 = str(row['context_2']) if not pd.isna(row['context_2']) else ''
        context_1 = str(row['context_1']) if not pd.isna(row['context_1']) else ''
        response = str(row['response']) if not pd.isna(row['response']) else ''

        text_sequence = context_3 + "\n" + context_2 + "\n" + context_1 + "\n" + response
        file.write(text_sequence + "\n")

from transformers import LineByLineTextDataset
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text_data.txt",
    block_size=128,
)

training_args = TrainingArguments(
    output_dir="./output_directory",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=32,
    save_steps=10_000,
    evaluation_strategy="steps",
    eval_steps=10_000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()

model.save_pretrained("./saved_model")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
